In [ ]:
import pandas as pd
import numpy as np
import pandera as pa
import pandera.extensions as extensions
from pandera.typing import DataFrame, Series, Index, DateTime
from typing import Callable

import jwt
import requests
import json

### API Parameters

In [ ]:
iss = ""
aud = "erp-core.brusnika.ru/parameters"
key = ""

payload = {
    "iss": iss,
    "aud": aud,
}

token_params = jwt.encode(payload,key)

In [ ]:
working_version = '1ce7fe21-956f-4627-b1aa-cbfc2d6358ba'

In [ ]:
obj_urls = [
    f'https://erp-core.brusnika.ru/parameters/v1/versions/{working_version}/construction-objects',
    f'https://erp-core.brusnika.ru/parameters/v1/versions/{working_version}/infrastructure-objects',
    f'https://erp-core.brusnika.ru/parameters/v1/versions/{working_version}/transient-objects'
]

In [ ]:
resp_params_objs = []
for url in obj_urls:
    resp = requests.get(url, headers={"authorization":f"Bearer {token_params}"})
    resp_params_objs.extend(resp.json())

In [ ]:
resp_params_constr_obj_df = pd.DataFrame(resp_params_objs)

In [ ]:
cols_for_preproc = resp_params_constr_obj_df.columns[
    resp_params_constr_obj_df.applymap(
        lambda x: isinstance(x, dict) and x.get('id') is not None
    ).all(axis=0)
]

for col in cols_for_preproc:
    resp_params_constr_obj_df[col] = resp_params_constr_obj_df[col].apply(lambda x: x.get('id'))

In [ ]:
resp_params_constr_obj_df = resp_params_constr_obj_df.set_index('id')

In [ ]:
not_canceled_constr_obj = resp_params_constr_obj_df.index[~resp_params_constr_obj_df.canceled]
not_canceled_constr_obj.name = 'constructionObject'

Список актуальных для проверки объектов:

* из ПАРАМЕТРИЧЕСКОЙ системы
* не отмененные (`canceled = False`)

### API Processes

In [ ]:
iss = ""
aud = "erp-core.brusnika.ru/processes"
key = ""

payload = {
    "iss": iss,
    "aud": aud,
}

token_process = jwt.encode(payload,key)

In [ ]:
tasks = {
    "88359fe3-4d1a-4bac-8fee-9ddd32eb9dc1": "permission_date", #РС
    "8b1663d9-a69d-40fd-8b3e-9f51b39d66a8": "construction_start_date", # НС
    "733fb7d0-0ec6-4223-846e-dcc0bebaaca9": "commition_date", #РВ
}

In [ ]:
url = f'https://erp-core.brusnika.ru/processes/v2/versions/{working_version}/process-statistics'

request_body ={
    "objectFilter": [
        {
            "projections": [],
            "stages": None,
            "processes": None,
            "tasks": None,
            "contours": None,
            "projects": None,
            "sites": None,
            "parcels": None,
            "constructionObjects": [],
            "objectTypes": ["CONSTRUCTION_OBJECT"]
        }
    ],
    "periodFilter": {
        "since": None,
        "until": None
    },
    "groupBy": "TASK"
}

In [ ]:
resp_process_dates_json = {}

for task_id in tasks.keys():
    request_body['objectFilter'][0].update({"tasks": [task_id]})
    resp_process_date = requests.post(url, headers={"authorization":f"Bearer {token_process}"}, json=request_body)
    resp_process_dates_json.update({task_id: resp_process_date.json()})
    
del resp_process_date

In [ ]:
columns = [
    # 'overallStart', 
    'overallEnd', 
    'constructionObject', 
    # 'modeled', 
    'status',
]

In [ ]:
resp_process_dates_dfs = {}

for task_id in resp_process_dates_json.keys():
    df = pd.DataFrame(resp_process_dates_json[task_id])[columns]
    
    df.loc[:, 'constructionObject'] = df.constructionObject.apply(lambda x: x.get('id'))
    df = df.set_index('constructionObject')
    
    resp_process_dates_dfs.update({task_id: df})

del df

In [ ]:
for task_id in resp_process_dates_dfs.keys():
    # если надо более очевидно, то нужно будет сделать копии
    df = resp_process_dates_dfs[task_id] # указатель на сет в словаре
    task_name = tasks[task_id]
    df.columns = task_name + '_' + df.columns # потому сразу меняется в словаре

In [ ]:
combine_process_dates_df = pd.concat(resp_process_dates_dfs.values(), axis=1)

In [ ]:
combine_process_dates_df.shape

In [ ]:
combine_process_dates_df.head(3)

Объединяем знания о не отмененных объектах и данные процессной системы

In [ ]:
common_df = pd.DataFrame(index = not_canceled_constr_obj)
common_df = common_df.join(combine_process_dates_df, how='left')

In [ ]:
common_df.head(3)

In [ ]:
common_df.shape

### API Indicators

In [ ]:
iss = ""
aud = "erp-core.brusnika.ru/indicators"
key = ""

payload = {
    "iss": iss,
    "aud": aud,
}

token_indicators = jwt.encode(payload,key)

Поиск последней утвержденной версии релиза

In [ ]:
url = 'https://erp-core.brusnika.ru/indicators/api/v2/versions'
resp_ind_versions = requests.get(url, headers={"authorization":f"Bearer {token_indicators}"})
resp_ind_versions_df = pd.DataFrame(resp_ind_versions.json())
resp_ind_versions_release = resp_ind_versions_df[resp_ind_versions_df.title.str.startswith('Релиз')]

**!!!** тут у меня другой ключ

In [ ]:
iss = ""
aud = "erp-core.brusnika.ru/versioning"
key = ""

payload = {
    "iss": iss,
    "aud": aud,
}

token_version = jwt.encode(payload,key)

In [ ]:
url = 'https://scheduler.brusnika.ru/backend/versioning/v2/versions'
resp_versioning = requests.get(url, headers={"authorization":f"Bearer {token_version}"})
resp_versioning

In [ ]:
resp_versioning_df = pd.DataFrame(resp_versioning.json())
resp_versioning_df = resp_versioning_df.set_index('id')[['title', 'state', 'createdAt']]
resp_versioning_df_release = resp_versioning_df.loc[resp_ind_versions_release.id]
resp_versioning_df_release.createdAt = pd.to_datetime(resp_versioning_df_release.createdAt.str.split('T', expand=True).iloc[:, 0])

resp_versioning_df_release = resp_versioning_df_release[resp_versioning_df_release.state == 'APPROVED'].sort_values('createdAt')
release_version = resp_versioning_df_release.iloc[-1].name

In [ ]:
# release_version = "4713f87f-598f-43ec-866b-3a52111b0d92"

In [ ]:
url = f'https://erp-core.brusnika.ru/indicators/api/v2/versions/{release_version}/indicator-values'
request_body = [{
    "requestId": "",
    "indicators": [
        "po.production.total"
    ],
    "timeSlice": {
        "groupBy": "MONTH",
        "filter": {}
    },
    "objectSlice": {
        "groupBy": [
            "constructionObjectId",
            "itemTreeId"
        ],
        "filter": [
        {
            "constructionObjectId": [],
            "siteId": [],
            "projectId": [],
            "contourId": [],
            "premisesType": [],
            "fundsSourceType": [],
            "grade": [],
            "itemTreeId": [
                "76630097-e1b1-4787-b509-36e4a13c1490",
                "9f90fee4-22b0-4c75-b298-03b9af46a679",
                
                "4734c36c-9c27-44bc-82a1-1265267b0737",
                "05cacdf3-9de7-4bae-8916-59be9ef6c3c2",
                "ce719b53-3bf5-4c30-b8e5-7a1c8feaa4cb",
                "17d417b9-f6f1-44c2-b554-830e7a903274",
                "d3034170-b66f-4914-a2fd-e2cbc986038b",
                "e3bbb51d-07a1-44a0-a628-a1504f87ffe9",
            ]
        }
      ]
    }
    }
]

resp_ind_prod_total = requests.post(url, headers={"authorization":f"Bearer {token_indicators}"}, json=request_body)
resp_ind_prod_total

In [ ]:
resp_ind_prod_total_df = pd.DataFrame(resp_ind_prod_total.json()[0]['slices'])
resp_ind_prod_total_df.yearMonth = pd.to_datetime(resp_ind_prod_total_df.yearMonth)

In [ ]:
resp_ind_prod_total_df_first_date = resp_ind_prod_total_df[resp_ind_prod_total_df.amount > 0]
resp_ind_prod_total_df_first_date = resp_ind_prod_total_df_first_date.sort_values(by=['constructionObjectId', 'yearMonth'], ascending=True)
resp_ind_prod_total_df_first_date = resp_ind_prod_total_df_first_date.groupby('constructionObjectId').first()

resp_ind_prod_total_df_first_date = resp_ind_prod_total_df_first_date.drop(columns='itemTreeId')
resp_ind_prod_total_df_first_date.columns = 'first_spend_' + resp_ind_prod_total_df_first_date.columns

In [ ]:
resp_ind_prod_total_df_first_date.head(3)

Объединяем знания о не отмененных объектах + проц.с. с данными показательной системы

In [ ]:
common_df = common_df.join(resp_ind_prod_total_df_first_date, how='left')

In [ ]:
common_df.head(3)

### ВСЕ ПРОВЕРКИ

In [ ]:
def compare_two_dates(first_date: pd.Series, second_date: pd.Series, compare_func: Callable=pd.Series.gt, threshold: int=0, exclude_na: bool=True) -> pd.Series:
    """Сompares two series of dates by finding the difference between them, 
    and the difference with a threshold value.

    compare_func: pd.Series.gt, pd.Series.ge, pd.Series.lt or pd.Series.le
    threshold: treshold value in days
    """
    diff = second_date - first_date
    result = compare_func(diff, pd.Timedelta(days=threshold))
    if exclude_na:
        na_indexes = (first_date.isna() | second_date.isna())
        return result | na_indexes
    return result

In [ ]:
@extensions.register_check_method()
def check_permission_date_less_construction_start_date(df):
    return compare_two_dates(df['permission_date_overallEnd'], df['construction_start_date_overallEnd'], pd.Series.gt, threshold=0, exclude_na=True)

@extensions.register_check_method()
def check_diff_commition_date_construction_start_date_gt_year(df):
    return compare_two_dates(df['construction_start_date_overallEnd'], df['commition_date_overallEnd'], pd.Series.gt, threshold=365, exclude_na=True)

@extensions.register_check_method()
def check_diff_commition_date_permission_date_gt_year_lt_3_5_years(df):
    gt_year = compare_two_dates(df['permission_date_overallEnd'], df['commition_date_overallEnd'], pd.Series.gt, threshold=365, exclude_na=True)
    lt_3_5_years = compare_two_dates(df['permission_date_overallEnd'], df['commition_date_overallEnd'], pd.Series.lt, threshold=1279, exclude_na=True)
    return gt_year & lt_3_5_years

In [ ]:
@extensions.register_check_method()
def check_first_spend_is_in_construction_start_month(df, *, exclude_na: bool = True):
    start_date = df.construction_start_date_overallEnd
    spend_date = df.first_spend_yearMonth
    
    result = (start_date.dt.month == spend_date.dt.month) & (start_date.dt.year == spend_date.dt.year)  # check
    # is_complete = (df.commition_date_status == 'COMPLETE')
    # result = check | is_complete

    if exclude_na:
        na_indexes = (start_date.isna() | spend_date.isna())
        return result | na_indexes
    return result

In [ ]:
class Schema_milestone_dates(pa.DataFrameModel):
    permission_date_overallEnd: Series[DateTime] = pa.Field(nullable=False)
    construction_start_date_overallEnd: Series[DateTime] = pa.Field(nullable=False)
    commition_date_overallEnd: Series[DateTime] = pa.Field(nullable=False)
    # commition_date_status: Series[str] = pa.Field(nullable=True)
    first_spend_yearMonth: Series[DateTime] = pa.Field(nullable=True)
    first_spend_amount: Series[float] = pa.Field(nullable=True) # без проверок, и так уже после обработки, можно вообще убрать это поле наверное
    constructionObject: Index[str] = pa.Field(isin=not_canceled_constr_obj, check_name=True, unique=True)
    
    class Config:
        coerce = True

        check_permission_date_less_construction_start_date = ()
        check_diff_commition_date_construction_start_date_gt_year = ()
        check_diff_commition_date_permission_date_gt_year_lt_3_5_years = ()
        check_first_spend_is_in_construction_start_month = ()

In [ ]:
error = None
try:
    Schema_milestone_dates.validate(common_df, lazy=True)
except pa.errors.SchemaErrors as err:
    error = err
    display(err.failure_cases)

оставляем такой вывод, нужна логика получения результатов

In [ ]:
check_errors = error.failure_cases.groupby(by=['check'])['index'].unique().to_dict()
check_errors.keys()

In [ ]:
error.data.loc[check_errors['check_diff_commition_date_permission_date_gt_year_lt_3_5_years']]

In [ ]:
not_nullable_dict = error.failure_cases[error.failure_cases.check == 'not_nullable'].groupby('column')['index'].unique().to_dict()

In [ ]:
common_df.loc[
    error.failure_cases[error.failure_cases.check == 'check_diff_commition_date_permission_date_gt_year_lt_3_5_years']['index'].unique()
]

In [ ]:
common_df.shape

In [ ]:
for check, bad_id_list in check_errors.items():
    print(check, len(bad_id_list))

In [ ]:
for check, bad_id_list in not_nullable_dict.items():
    print(check, len(bad_id_list))

In [ ]:
common_df.construction_start_date_overallEnd.isna().sum()